In [2]:
import cv2
import numpy as np

In [25]:
last_known_left_line = None     # 上一幀的記憶(左線)
last_known_right_line = None    # 上一幀的記憶(右線)

# 建立梯形遮罩
def region_of_interest(image):

    height = image.shape[0]
    width = image.shape[1]
    
    # 建立梯形區域
    polygons = np.array([
        [(int(width*0.18), height),             # 左下 
         (int(width*0.48), int(height*0.73)),   # 左上 
         (int(width*0.5), int(height*0.73)),    # 右上 
         (int(width*0.7), height)]              # 右下 
    ], dtype=np.int32)
    
    mask = np.zeros_like(image)                     # 建立全黑的遮罩
    cv2.fillPoly(mask, polygons, 255)               # 將多邊形區域填滿白色
    masked_image = cv2.bitwise_and(image, mask)     # 只保留遮罩區域內的邊緣

    return masked_image, polygons

def make_coordinates(image, line_parameters):
   
    # 將斜率和截距轉換回影像中的 x, y 座標。
    
    slope, intercept = line_parameters
    y1 = image.shape[0]      
    y2 = int(y1 * 0.75)     
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    
    # Hough轉換偵測到的多條線段，平均成左右兩條車道線

    left_fit = []  # 左車道線的斜率、截距
    right_fit = [] # 右車道線的斜率、截距
    
    if lines is None:
        return None, None
        
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        
        # 忽略垂直線
        if x1 == x2:
            continue

        # 計算斜率和截距
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        
        if slope < -0.3: # 負斜率 (左側)
            left_fit.append((slope, intercept))
        elif slope > 0.3: # 正斜率 (右側)
            right_fit.append((slope, intercept))
            
    left_line = None
    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
        left_line = make_coordinates(image, left_fit_average)
        
    right_line = None
    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        right_line = make_coordinates(image, right_fit_average)
        
    return left_line, right_line

def calculate_and_draw_deviation(image, left_line, right_line, roi_polygon_overlay):
   
    # 計算並畫出偏離中線指示
   
    height, width, _ = image.shape
    
    # 1. 找到車道線在影像底部的 x 座標
    left_x_bottom = left_line[0]
    right_x_bottom = right_line[0]
    
    lane_center_x = (left_x_bottom + right_x_bottom) / 2
    image_center_x = width / 2

    # 計算偏移量
    offset_pixels = image_center_x - lane_center_x
    
    # 轉換為公尺，假設車道寬3.7m
    lane_width_pixels = right_x_bottom - left_x_bottom
    if lane_width_pixels != 0:
        xm_per_pix = 3.7 / lane_width_pixels
        offset_meters = offset_pixels * xm_per_pix
        direction = "left" if offset_meters > 0 else "right"
        text = f"Offset: {abs(offset_meters):.2f}m to the {direction}"
    else:
        text = "Calculating..."

    cv2.putText(image, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.line(roi_polygon_overlay, (int(lane_center_x), height), (int(lane_center_x), height - 40), (255, 0, 255), 8)
    
    return image, roi_polygon_overlay

def draw_lane(original_image, processed_image, left_line, right_line, roi_pts):
    
    # 建立一個空白畫布
    overlay = np.zeros_like(original_image)
    # 畫出綠色車道區域 
    if left_line is not None and right_line is not None:
        pts = np.array([[left_line[0], left_line[1]], [left_line[2], left_line[3]], 
                        [right_line[2], right_line[3]], [right_line[0], right_line[1]]], np.int32)
        cv2.fillPoly(overlay, [pts], (0, 255, 0)) 

        # 計算中心線的底部和頂部座標
        center_x_bottom = (left_line[0] + right_line[0]) // 2
        center_y_bottom = left_line[1] 
        center_x_top = (left_line[2] + right_line[2]) // 2
        center_y_top = left_line[3] 

        # 畫一條黃色實線作為中心線
        cv2.line(overlay, (center_x_bottom, center_y_bottom), (center_x_top, center_y_top), (0, 255, 255), 2) # 黃色, 2px
      
        # 偏離指示
        original_image, overlay = calculate_and_draw_deviation(original_image, left_line, right_line, overlay)

    # 合併綠色區域與原圖 (可調整透明度)
    final_image = cv2.addWeighted(original_image, 0.7, overlay, 0.4, 0)
    
    # 畫出車道線 
    if left_line is not None:
        cv2.line(final_image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (125, 0, 255), 5)
    if right_line is not None:
        cv2.line(final_image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (125, 0, 255), 5)

    return final_image


def process_frame(frame):
  
    global last_known_left_line
    global last_known_right_line

    
    original_frame = np.copy(frame)                             # 複製原始幀
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)              # 灰階化
    kernel_dilate = np.ones((3, 3), np.uint8)                   # 影像擴張
    dilated = cv2.dilate(gray, kernel_dilate, iterations=1)
    blur = cv2.GaussianBlur(dilated, (5, 5), 0)                 # 高斯濾波器減少雜訊
    canny = cv2.Canny(blur, 50, 150)                            # Canny 邊緣偵測
    masked_canny, roi_polygon_pts = region_of_interest(canny)   # 針對Canny邊緣偵測結果進行影像遮罩

    #  Hough Transformation (影像, 精度, 精度, 閾值, 最小線長, 最大線間距)
    lines = cv2.HoughLinesP(masked_canny, rho=2, theta=np.pi/180, threshold=100, minLineLength=40, maxLineGap=5)

   # 平均擬合線段
    left_line, right_line = average_slope_intercept(frame, lines) 
    
    # 是否偵測到左線 (有：更新 / 沒有：使用上一幀結果)
    if left_line is None:                       
        left_line = last_known_left_line  
    else:
        last_known_left_line = left_line

    # 是否偵測到右線 (有：更新 / 沒有：使用上一幀結果)
    if right_line is None:                      
        right_line = last_known_right_line
    else:
        last_known_right_line = right_line
        
    final_image = draw_lane(original_frame, masked_canny, left_line, right_line, roi_polygon_pts)

    return final_image

video_path = 'LaneVideo.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: 無法開啟影片檔案 {video_path}")
    exit()

cv2.namedWindow('Lane Detection (Press Q to quit)', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Lane Detection (Press Q to quit)', 1280, 720)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("影片播放完畢或讀取錯誤。")
        break

    processed_image = process_frame(frame)
    cv2.imshow('Lane Detection (Press Q to quit)', processed_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()